## Evaluation code

In [1]:
from io_ import readFiles
import numpy as np

# This is a sample test folder
test_folder = 'test_images'
image_paths = readFiles(test_folder)
print('Below are test images with their ground_truth class index appeneded:')
print(image_paths)

Below are test images with their ground_truth class index appeneded:
['a_0.jpg' 'b_21.jpg' 'c_7.JPG' 'd_4.jpg' 'e_22.jpg']


## Score each of the images based on the likelihood it belongs to a class. Larger scores indicate higher likelihood.

Classes are arranged alphabeticall, [1_Capitol, 1_Chijmes,..., 1_scis, americano, ... taco]. Screen-shot is attached.

The first thirteen belong to buidings and the next fifty nine belong to food, this makes for a total of 72 classes.

Scores should take the form of a [Nx72] array, where N is the number of test images.


In [2]:
################# Code for sorting the classes (not needed for project) ##################

# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import numpy as np

# train_dir = '../training'



# train_datagen = ImageDataGenerator() 
# train_generator = train_datagen.flow_from_directory(
#     train_dir) 

# a = train_generator.class_indices
# l = []
# for i in a:
#     l.append(i)
# np.save('sorted_classes.npy', l)

In [2]:
sorted_class = np.load('sorted_classes.npy')
print(sorted_class[0], ' is class ', 0)
print(sorted_class[10], ' is class ', 10)
print(sorted_class[21], ' is class ', 21)

1_Capitol  is class  0
1_library  is class  10
cafe_coffee_with_milk  is class  21


In [6]:
############## Write your test function here ##############


# Below is a possible mechanics for score computation;
# However, it must be modifed according to pre-processing amd network


# from tensorflow.keras.preprocessing import image
# num_images = len(image_paths)
# scores = np.zeros([num_images, 72])
# for i,im in enumerate(image_paths):
#     img = image.load_img(test_folder + '/' + im, target_size=(224, 224))
#     img = image.img_to_array(img)
#     img = img.reshape([1,224,224,3])
#     scores[i] = model.predict(img)


In [3]:
CLASS_NAMES = sorted_class.tolist()

In [4]:
############## Write your test function here ##############
import json
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam

# Below is a possible mechanics for score computation;
# However, it must be modifed according to pre-processing amd network

In [5]:
# Load model and weights from json and h5 files
MODEL_FILE = os.path.join('..', 'GerryChng_EfficientNetB5_20210414_01.json')
MODEL_WEIGHTS = os.path.join('..', 'GerryChng_EfficientNetB5_20210414_01.h5')

# Model reconstruction from JSON file
with open(MODEL_FILE, 'r') as json_file:
    json_savedModel= json_file.read()

test_model = tf.keras.models.model_from_json(json_savedModel)
test_model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['acc'])

# Load weights into the new model
test_model.load_weights(MODEL_WEIGHTS)

In [ ]:
test_model.summary()

In [6]:
# Define my own ground truth array based on the actual directory the file was from
my_test_gt = [int(os.path.splitext(x)[0].split('_')[-1]) for x in image_paths]

# Define my own ground truth array based on the actual directory the file was from
# image_classes = [x.split('___')[0] for x in image_paths]
# my_test_gt = [CLASS_NAMES.index(x) for x in image_classes]

In [7]:
num_images = len(image_paths)

# Training image dimensions - 456x456 because we are loading EfficientNetB5
IMG_SIZE = 456
IMG_CHANNELS = 3

scores = np.zeros([num_images, 72])
image_file = [None] * num_images

for i,im in enumerate(image_paths):
    img = image.load_img(test_folder + '/' + im, target_size=(456, 456))
    img = image.img_to_array(img)
    img = img.reshape([1,456,456,3])
    scores[i] = test_model.predict(img)
    image_file[i] = im


In [8]:
results = np.argmax(scores[:20], axis=1)
print(results)

[ 3 21  7  4 22]


## Demonstrating the evaluation functions on the first two test images

You need not modify code beyond this point.

In [9]:
from io_ import score
print('overall accuracy')

# Ground-truth: the first image belongs to class 0; the second image to class 13.
# gt = np.array([0,21]) 
gt = np.array(my_test_gt)

top1, top5 = score(scores, gt, 5)
print('percentage top1 accuracy:', top1)
print('percentage top5 accuracy:', top5)

overall accuracy
percentage top1 accuracy: 0.8
percentage top5 accuracy: 1.0


In [10]:
print('accuracy on building only')
mask = gt<13
top1, top5 = score(scores[mask,:13], gt[mask], 5)
print('percentage top1 accuracy:', top1)
print('percentage top5 accuracy:', top5)

accuracy on building only
percentage top1 accuracy: 0.6666666666666666
percentage top5 accuracy: 1.0


In [11]:
print('accuracy on food only')
mask = gt>=13
top1, top5 = score(scores[mask,13:], gt[mask]-13, 5)
print('percentage top1 accuracy:', top1)
print('percentage top5 accuracy:', top5)

accuracy on food only
percentage top1 accuracy: 1.0
percentage top5 accuracy: 1.0
